**Упражнение 1.** Запустите код. Попробуйте объяснить, почему LIST - пуст.

In [1]:
import multiprocessing

def worker():
    LIST.append('item')

LIST = []

if __name__ == "__main__":
    processes = [
        multiprocessing.Process(target=worker)
        for _ in range(5)
    ]
    for p in processes:
        p.start()
    for p in processes:
        p.join()
    print(LIST)

[]


Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/opt/homebrew/Cellar/python@3.10/3.10.9/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/opt/homebrew/Cellar/python@3.10/3.10.9/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'worker' on <module '__main__' (built-in)>
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/opt/homebrew/Cellar/python@3.10/3.10.9/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/opt/homebrew/Cellar/python@3.10/3.10.9/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.l

В данном примере, переменная LIST создана в главном процессе и не передается в созданные процессы, поэтому каждый процесс работает с отдельной копией переменной LIST. Когда каждый процесс пытается добавить item в свою копию списка, на самом деле они изменяют разные списки, которые не связаны друг с другом. Поэтому, после завершения всех процессов, список в главном процессе остается пустым.

**Упражнение 2.** Запустите код. Какая проблема здесь возникает? Исправьте её.

In [2]:
import threading
import time

l1 = threading.Lock()
l2 = threading.Lock()

def f1(name):
    print('thread',name,'about to lock l1')
    with l1:
        print('thread',name,'has lock l1')
        time.sleep(0.3)
        print('thread',name,'about to lock l2')
        with l2:
            print('thread',name)

def f2(name):
    print('thread',name,'about to lock l2')
    with l1:
        print('thread',name,'has lock l2')
        time.sleep(0.3)
        print('thread',name,'about to lock l1')
        with l2:
            print('thread',name)

if __name__ == '__main__':
    t1=threading.Thread(target=f1, args=['t1',])
    t2=threading.Thread(target=f2, args=['t2',])

    t1.start()
    t2.start()

    t1.join()
    t2.join()

threadthread t2 about to lock l2
thread t2 has lock l2
 t1 about to lock l1
thread t2 about to lock l1
thread t2
thread t1 has lock l1
thread t1 about to lock l2
thread t1


Данный код содержит deadlock (взаимную блокировку) - ситуацию, когда два или более потока ждут друг друга, чтобы получить доступ к ресурсу, заблокированному другим потоком. В этом случае два потока пытаются получить блокировки на двух разных объектах l1 и l2 в разном порядке, что может привести к блокировке одного или обоих потоков.

В функции f1 поток сначала блокирует l1, а затем l2, в то время как в функции f2 поток блокирует l2, а затем l1. Если поток f1 заблокировал l1 и ждет доступа к l2, а поток f2 заблокировал l2 и ждет доступа к l1, то оба потока будут заблокированы, и программа зависнет.

**Упражнение 3.** Вам необходимо вычислить значение функции f = x**2 + x * 2 + 10 * x для различных значений аргументов (10 случайных чисел).

1. Сделайте программу без использования потоков/процессов и измерьте время.

2. Разбейте задачу на несколько потоков (отдельный поток для каждого аргумента и отдельный поток для каждого слагаемоего функции). Запустите программу. Измерьте время. 

3. Разбейте задачу на несколько процессов (отдельный процесс для каждого аргумента и отдельный процесс для каждого слагаемоего функции). Запустите программу. Измерьте время. 

Для синхронизации можно использовать барьеры.

Объясните полученные результаты.

In [5]:
import random
import time

def function(x):
    return x**2 + x*2 + 10*x


prev_time = time.time()
for i in range(10):
    x = random.randint(-100, 100)
    result = function(x)
    print(f"f({x}) = {result}")
print(time.time() - prev_time)


f(62) = 4588
f(-64) = 3328
f(-2) = -20
f(2) = 28
f(-79) = 5293
f(-32) = 640
f(77) = 6853
f(-52) = 2080
f(56) = 3808
f(-85) = 6205
0.00032210350036621094


In [8]:
import threading

def calculate_sum(x, result_list):
    result = x**2
    result_list[0] = result

def calculate_product(x, result_list):
    result = x*2
    result_list[1] = result

def calculate_constant(x, result_list):
    result = 10*x
    result_list[2] = result
    
prev_time = time.time()

def calculate_function(x, result_list):
    threads = []
    results = [None, None, None]
    
    threads.append(threading.Thread(target=calculate_sum, args=(x, results)))
    threads.append(threading.Thread(target=calculate_product, args=(x, results)))
    threads.append(threading.Thread(target=calculate_constant, args=(x, results)))
    
    for thread in threads:
        thread.start()
    
    for thread in threads:
        thread.join()
    
    result = sum(results)
    result_list.append(result)

results = []

threads = []
for x in range(10):
    result_list = []
    threads.append(threading.Thread(target=calculate_function, args=(x, result_list)))
    results.append(result_list)

for thread in threads:
    thread.start()

for thread in threads:
    thread.join()

for i, result_list in enumerate(results):
    result = result_list[0]
    print(f"f({i}) = {result}")

    
print(time.time() - prev_time)


f(0) = 0
f(1) = 13
f(2) = 28
f(3) = 45
f(4) = 64
f(5) = 85
f(6) = 108
f(7) = 133
f(8) = 160
f(9) = 189
0.0064868927001953125


In [ ]:
import multiprocessing

def calculate_sum(x, result_list):
    result = x**2
    result_list[0] = result

def calculate_product(x, result_list):
    result = x*2
    result_list[1] = result

def calculate_constant(x, result_list):
    result = 10*x
    result_list[2] = result

def calculate_function(x, result_list):
    processes = []
    results = multiprocessing.Array('i', 3)
    
    processes.append(multiprocessing.Process(target=calculate_sum, args=(x, results)))
    processes.append(multiprocessing.Process(target=calculate_product, args=(x, results)))
    processes.append(multiprocessing.Process(target=calculate_constant, args=(x, results)))
    
    for process in processes:
        process.start()
    
    for process in processes:
        process.join()

    result = sum(results)
    result_list.append(result)

if __name__ == '__main__':
    prev_time = time.time()
    results = []
    processes = []
    for x in range(10):
        result_list = multiprocessing.Manager().list()
        processes.append(multiprocessing.Process(target=calculate_function, args=(x, result_list)))
        results.append(result_list)

    for process in processes:
        process.start()

    for process in processes:
        process.join()

    for i, result_list in enumerate(results):
        result = result_list[0]
        print(f"f({i}) = {result}")
    print(time.time() - prev_time)


В данном примере быстрее всего отработает однопоточный код, тк используются несложные вычисления, а накладки на создания потоков и процессов слишком высоки

**Упражнение 4.** Смоделируйте следующую ситуацию с использованием семафоров и событий.

1. Есть 5 касс и 20 покупателей. Все они хотят купить билеты на матч. На билетах не указаны места. 

2. После покупки билета покупатель бежит на стадион (у каждого разная скорость бега, это намек, что надо сделать sleep) и занимает свободное место.

Программа должна вывести логированные события по типу:

client 0, service time (ticket): 1.0004174709320068

client 1, service time (ticket): 1.0005174709320068

client 1, runnig time to stad:   1.2004174709320068

client 0, runnig time to stad:   1.5004174709320068

In [ ]:
import threading
import time

# Создаем семафоры и события
free_cashiers = threading.Semaphore(5)
ticket_sold = threading.Event()
seat_taken = threading.Event()
free_seats = threading.Semaphore(20)


# Определяем класс покупателя
class Customer(threading.Thread):
    def __init__(self, name):
        super().__init__()
        self.name = name

    def run(self):
        # Покупаем билет
        print(f"client {self.name}, service time (ticket): {time.time()}")
        time.sleep(1)

        # Бежим на стадион и ищем свободное место
        print(f"client {self.name}, running time to stad: {time.time()}")
        time.sleep(2)
        free_seats.acquire()

        # Освобождаем кассу и сигнализируем о занятии места
        free_cashiers.release()
        seat_taken.set()


# Создаем 20 покупателей и запускаем их в работу
for i in range(20):
    # Ждем свободной кассы
    free_cashiers.acquire()
    c = Customer(name=f"{i}")
    c.start()

# Кассир продает билеты и освобождает место на стадионе
for _ in range(20):
    print(f"service time (ticket): {time.time()}")
    ticket_sold.set()
    seat_taken.wait()
    ticket_sold.clear()


**Упражнение 5.** Покажите и опишите разничу между Semaphore и BoundedSemaphore.

Semaphore и BoundedSemaphore - это два класса из модуля multiprocessing, которые используются для контроля доступа к общим ресурсам в многопоточных или многопроцессорных средах.

Semaphore - это объект, который управляет доступом к ресурсу путем подсчета числа разрешений на доступ. Когда поток хочет получить доступ к ресурсу, он запрашивает разрешение. Если есть доступные разрешения, то поток занимает ресурс, а число разрешений уменьшается. Если все разрешения заняты, то поток блокируется и ждет, пока один из других потоков не освободит ресурс. Пример

In [ ]:
from multiprocessing import Semaphore, Process
import time

def worker(s, i):
    s.acquire()
    print(f"Worker {i} is working")
    time.sleep(1)
    s.release()

if __name__ == '__main__':
    s = Semaphore(2)
    processes = []
    for i in range(5):
        p = Process(target=worker, args=(s, i))
        processes.append(p)
        p.start()

    for p in processes:
        p.join()


В этом примере мы создаем объект Semaphore(2), что означает, что два потока могут получить доступ к ресурсу одновременно. Мы запускаем 5 процессов, которые пытаются получить доступ к ресурсу. Первые два процесса получают доступ к ресурсу, тогда как остальные блокируются и ждут, пока один из первых двух процессов освободит ресурс.

BoundedSemaphore - это подкласс Semaphore, который также управляет доступом к ресурсу, но имеет ограниченный размер, который задается при создании объекта. Это гарантирует, что количество разрешений на доступ никогда не будет превышать заданного значения. Пример:

In [ ]:
from multiprocessing import BoundedSemaphore, Process
import time

def worker(s, i):
    s.acquire()
    print(f"Worker {i} is working")
    time.sleep(1)
    s.release()

if __name__ == '__main__':
    s = BoundedSemaphore(2)
    processes = []
    for i in range(5):
        p = Process(target=worker, args=(s, i))
        processes.append(p)
        p.start()

    for p in processes:
        p.join()


В этом примере мы создаем объект BoundedSemaphore(2), что означает, что только два процесса могут получить доступ к ресурсу одновременно. Мы запускаем 5 процессов, которые пытаются получить доступ к ресурсу. Первые два процесса получают доступ к ресурсу, выполняют свою работу, освобождают ресурс, и затем следующие два процесса получают доступ к ресурсу, и т.д. Таким образом, только два процесса могут работать с ресурсом в данный момент.

**Упражнение 6.** Запустите на исполнение, замерив время работы. Перепишите с помощью потоков и опять замерьте время. Затем с помощью процессов и снова измерьте время. Объясните результат.

In [ ]:
import urllib.request
import time


urls = [
    'https://www.yandex.ru', 'https://www.google.com',
    'https://habrahabr.ru', 'https://www.python.org',
    'https://isocpp.org',
]


def read_url(url):
    with urllib.request.urlopen(url) as u:
        return u.read()


start = time.time()
for url in urls:
    read_url(url)
print(time.time() - start)

4.498141050338745


In [13]:
import urllib.request
import time
import threading


urls = [
    'https://www.yandex.ru', 'https://www.google.com',
    'https://habrahabr.ru', 'https://www.python.org',
    'https://isocpp.org',
]


def read_url(url):
    with urllib.request.urlopen(url) as u:
        return u.read()


start = time.time()
threads = []
for url in urls:
    x = threading.Thread(target = read_url, args =(url,))
    threads.append(x)
    x.start()

for thread in threads:
    thread.join()
    
print(time.time() - start)

1.7011709213256836


In [ ]:
import urllib.request
import time
from multiprocessing import Process


urls = ['https://www.yandex.ru', 'https://www.google.com',    
        'https://habrahabr.ru', 'https://www.python.org',
        'https://isocpp.org',]


def read_url(url):
    with urllib.request.urlopen(url) as u:
        return u.read()


if __name__ == '__main__':
    start = time.time()
    processes = []

    for url in urls:
        p = Process(target=read_url, args=(url,))
        p.start()
        processes.append(p)

    for p in processes:
        p.join()

    print(time.time() - start)


Код с использованием потоков в данном случае будет работать быстрее, чем код с использованием процессов, так как размерность URL-адресов в нашем примере небольшая, так же, потоки в общем случае будут работать быстрее, тк потоки общаются между собой непосредственно и не требуют межпроцессного взаимодействия, что может замедлить работу. 